### Gram-Schmidt Orthonormalization for QR factorization

Given a matrix $A \in \mathbb{C}^{m\times n}$, we can express it as a row of vectors $\vec{a}_i$ 
\begin{equation}
    A = [\vec{a}_1, \vec{a}_2, ..., \vec{a}_n]
\end{equation}

There are two mathematically equivalent ways by which we can obtain the QR factorization of A. One is the classical Gram-Schmidt orthonormalization procedure, and the other is its modified version. Both versions lead to the same output when calculated by hand, but could give different numerical results due to roundoff error. You may consult these resources to learn more about these algorithms:

https://arnold.hosted.uark.edu/NLA/Pages/CGSMGS.pdf

In [1]:
# author: mugalino 2022

# Imports the necessary packages
import numpy as np

# This code computes for the QR factorization of m x n
# matrices with m >= n.

def matrix_norm(matrix, order=1):
    if order == 1:
        return max([np.sum(matrix[i,:]) for i in range(np.shape(matrix)[0])])
    elif order == 2:
        return max([np.sum(matrix[:,i]) for i in range(np.shape(matrix)[1])])
    elif order == "fro":
        return np.sqrt(np.sum(np.abs(matrix)**2))

def gramschmidt(matrix, modified=False, check = True):
    mat_shape = np.shape(matrix)
    
    # Check if the matrix has shape m x n where m >= n
    if mat_shape[0] >= mat_shape[1]:
        None
    else:
        return "[Shape error] This is only applicable to square (mxm) or tall matrices (m>=n)"
    
    # Classical
    if modified:
        Q = np.zeros(mat_shape)
        R = np.zeros((mat_shape[1],mat_shape[1]))

        for i in range(len(matrix)-1):
            if i == 0:
                R[i,i] = np.sqrt(np.dot(matrix[:,i],matrix[:,i]))
                Q[:,0] = matrix[:,0]/np.sqrt(np.dot(matrix[:,0], matrix[:,0])) 
            else:
                Q[:,i] = matrix[:,i]

                for j in range(i):
                    R[j,i] = np.dot(Q[:,j],matrix[:,i])
                    Q[:,i] -= R[j,i] * Q[:,j]

                R[i,i] = np.sqrt(np.dot(Q[:,i],Q[:,i]))
                Q[:,i] /= np.sqrt(np.dot(Q[:,i], Q[:,i]))

    # Modified
    else:
        Q = np.zeros(mat_shape)
        R = np.zeros((mat_shape[1],mat_shape[1]))
        V = np.zeros(mat_shape)

        V[:,:] = matrix[:,:]

        for i in range(len(matrix)-1):
            Q[:,i] = V[:,i] / np.sqrt(np.dot(V[:,i], V[:,i])) 

            for j in range(i+1,len(matrix)-1):
                R[i,j] = np.dot(Q[:,i],V[:,j])
                V[:,j] -= R[i,j] * Q[:,i]
            R[i,i] = np.sqrt(np.dot(V[:,i],V[:,i]))
    if check:
        numpy_Q, numpy_R = np.linalg.qr(matrix)
        
        # Three error prescriptions, ||Q - numpyQ||, ||R - numpyR||, ||QR - matrix||
        print("||Q-numpy(Q)||=", matrix_norm(np.abs(Q) - np.abs(numpy_Q), order="fro"))
        print("||R-numpy(R)||=", matrix_norm(np.abs(R) - np.abs(numpy_R), order="fro"))
        print("||QR-A||=", matrix_norm(np.dot(Q,R) - matrix, order="fro"))

    return Q, R

In [2]:
matrix = np.array([[1,2,3,4],[2,5,7,10],[3,8,9,4]]).T

In [3]:
gramschmidt(matrix)

||Q-numpy(Q)||= 1.8450674160308825e-15
||R-numpy(R)||= 3.219646771412954e-15
||QR-A||= 8.08254562088053e-16


(array([[ 0.18257419, -0.71562645, -0.22372097],
        [ 0.36514837,  0.22019275,  0.79900347],
        [ 0.54772256, -0.49543369,  0.22372097],
        [ 0.73029674,  0.44038551, -0.51136222]]),
 array([[ 5.47722558, 13.32791557, 11.31959952],
        [ 0.        ,  0.60553007, -3.08269854],
        [ 0.        ,  0.        ,  5.68890467]]))